In [ ]:
import requests
import uuid

In [ ]:
# 两个id初次可以从插件读取，在每次获取authcode时作为请求头的一部分，响应的cookies会更新这两个值
remid = ''
sid = ''

In [ ]:
# 用remid和sid获取access_token
res_access_token = requests.get(
    url="https://accounts.ea.com/connect/auth",
    params= {
        'client_id': 'ORIGIN_JS_SDK',
        'response_type': 'token',
        'redirect_uri': 'nucleus:rest',
        'prompt': 'none',
        'release_type': 'prod'
    },
    headers= {
        'Cookie': f'remid={remid};sid={sid}',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.193 Safari/537.36',
        'content-type': 'application/json'
    }
)
access_token = res_access_token.json()['access_token']

In [ ]:
# 更新sid和remid
def upd_remid_sid(res: requests.Response, remid, sid):
    res_cookies = requests.utils.dict_from_cookiejar(res.cookies)
    if 'sid' in res_cookies:
        sid = res_cookies['sid']
    if 'remid' in res_cookies:
        remid = res_cookies['remid']
    return remid, sid

In [ ]:
remid, sid = upd_remid_sid(res_access_token, remid, sid)

In [ ]:
# 获取authcode
res_authcode = requests.get(
    url="https://accounts.ea.com/connect/auth",
    params= {
        'client_id': 'sparta-backend-as-user-pc',
        'response_type': 'code',
        'release_type': 'none'
    },
    headers= {
        'Cookie': f'remid={remid};sid={sid}'
    },
    allow_redirects=False
)
# 这个请求默认会重定向，所以要禁用重定向，并且重定向地址里的code参数就是我们想要的authcode
authcode = str.split(res_authcode.next.path_url, "=")[1]

remid, sid = upd_remid_sid(res_authcode, remid, sid)

In [ ]:
# 用authcode获取sessionID
res_session = requests.post(
    url="https://sparta-gw.battlelog.com/jsonrpc/pc/api",
    json= {
        'jsonrpc': '2.0',
        'method': 'Authentication.getEnvIdViaAuthCode',
        'params': {
            'authCode': authcode,
            "locale": "zh-tw",
        },
        "id": str(uuid.uuid4())
    }
)
sessioID = res_session.json()['result']['sessionId']

In [ ]:
# 使用sessionID示例， 获取欢迎信息
res = requests.post(
    url="https://sparta-gw.battlelog.com/jsonrpc/pc/api",
    params={'X-GatewaySession': sessioID},
    json = {
        'jsonrpc': '2.0',
        'method': 'Onboarding.welcomeMessage',
        'params': {
            'game' : "tunguska",
            'minutesToUTC' : "-480"
        },
        "id": str(uuid.uuid4())
    }
)
res.json()

In [ ]:
sessioID

In [ ]:
res_session.content